# markdown.obisidian.personal.index_notes
> Functions for managing index notes one's Obsidian.md math vault.

In a Obsidian math vault, it is convenient to keep index notes, which list links to other index notes or standard information notes. 

The methods in this module
- create (standard information) notes in appropriate folders,
- set up the notes,
- add links of the notes to appropriate index notes
- indicate in the index note and the standard information note where the content of the information note originates from in the original text.

In [ ]:
#| default_exp markdown.obsidian.personal.index_notes

In [ ]:
#| export
import glob
import os
from os import PathLike
from pathlib import Path
import re
from typing import Union

from natsort import natsorted
from pathvalidate import sanitize_filename

from trouver.helper.files_and_folders import path_name_no_ext
from trouver.markdown.markdown.file import (
    MarkdownFile, MarkdownLineEnum
)
from trouver.markdown.markdown.heading import heading_title
from trouver.markdown.obsidian.links import (
    link_ranges_in_text, ObsidianLink, links_from_text
)
from trouver.markdown.obsidian.vault import (
    VaultNote, note_name_unique, note_path_by_name
)

In [ ]:
import tempfile
from unittest import mock
import shutil


from fastcore.test import *
from pathvalidate import validate_filename

from trouver.helper.files_and_folders import path_name_no_ext
from trouver.helper.tests import _test_directory

## Identifying subsections listed in index notes and subsection folders

In [ ]:
#| export
def subsections_listed_in_index_note(
        index_note: Union[VaultNote, str], # The index note
        vault: PathLike
        ) -> dict[Union[int, str], [dict, str]]: # The keys are 1. line numbers and 2. `'title'`. The values are dict and str (the blank str if root node), respectively.
    """
    Return subsections/subchapters as listed in the index note

    **See Also**
    
    - The `get_headings_tree` function of the `MarkdownFile` class.
    """
    vault = Path(vault)
    if isinstance(index_note, str):
        index_note = VaultNote(vault, name=index_note)
    mf_file = MarkdownFile.from_vault_note(index_note)
    return mf_file.get_headings_tree()

In [ ]:
text = r"""# 1. Some section title
- [[some_note]], Page 1
- [[some_note_2]], Page 2

# 2. Some other section title
- [[some_note_3]], Page 2
- [[some_note_4]], Page 3

# 3. Section 3
- [[some_note_5|an alias]], Page 3

# 4. Section 4
# 5. Section 5
"""

with mock.patch("trouver.markdown.markdown.file.open", mock.mock_open(read_data=text)):
    fake_vn = VaultNote(rel_path='fake_note.md', vault='')  # Think of this as a VaultNote object whose underlying file has `text` as its content.
    subsections_in_text = subsections_listed_in_index_note(fake_vn, vault='')
    expected_output = {
        'title': '',
        0: {'title': '# 1. Some section title'},
        4: {'title': '# 2. Some other section title'},
        8: {'title': '# 3. Section 3'},
        11: {'title': '# 4. Section 4'},
        12: {'title': '# 5. Section 5'}}
    test_eq(subsections_in_text, expected_output)

In [ ]:
#| export
def subsection_folders(
        index_note: Union[VaultNote, str], # The index note
        vault: PathLike,
        output_type: str, # `'absolute_path'`, `'relative_path'`, or `'name'`
        ) -> list[str]: # List of immediate subdirectories in the directory containing the index note.
    """
    Return subdirectories corresponding to subsections/subchapters, i.e.
    the folders in the same directory as the index note.

    The folders are arranged in the order specified by `natsorted`.
    """
    vault = Path(vault)
    if isinstance(index_note, str):
        index_note = VaultNote(vault, name=index_note)
    parent_directory = (vault / index_note.rel_path).parent
    # print(str(parent_directory))
    glob_result = natsorted(glob.glob(str(parent_directory) + '/**/'))
    if output_type == 'absolute_path':
        return glob_result
    elif output_type == 'relative_path':
        return [str(Path(dir).relative_to(vault)) for dir in glob_result]
    elif output_type == 'name':
        return [Path(dir).name for dir in glob_result]

In [ ]:
with (tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir):
    temp_vault = Path(temp_dir) / 'test_vault_2'
    shutil.copytree(_test_directory() / 'test_vault_2', temp_vault)

    two_chapter_reference_1_index_note = VaultNote(temp_vault, name='_index_2_chapter_reference_1')
    ss_folders = subsection_folders(two_chapter_reference_1_index_note, temp_vault, output_type='name')
    assert len(ss_folders) > 0

In [ ]:
mock_vault = Path('mock_absolute_path')
mock_path =  mock_vault / Path('mock_reference_folder') / Path('mock_chapter')
folders = [  #glob.glob would return the folders in this order, at least on Windows:
    '1 section',
    '10 section',
    '11 section',
    '2 section',
    '3 section',
    '4 section',
    '5 section',
    '6 section',
    '7 section',
    '8 section',
    '9 section']

mock_glob_return_value = [str(mock_path / folder) for folder in folders]

with mock.patch("__main__.glob.glob", return_value=mock_glob_return_value):
    mock_index_note = VaultNote(rel_path='_index_mock_chapter.md', vault= mock_vault)
    
    sample_output_absolute_path = subsection_folders(mock_index_note, mock_vault, output_type='absolute_path')
    test_shuffled(sample_output_absolute_path, mock_glob_return_value)
    test_eq(sample_output_absolute_path, natsorted(mock_glob_return_value))

    sample_output_relative_path = subsection_folders(mock_index_note, mock_vault, output_type='relative_path')
    expected_output_for_relative_paths = [os.path.relpath(folder, mock_vault) for folder in mock_glob_return_value]
    test_shuffled(sample_output_relative_path, expected_output_for_relative_paths)
    test_eq(sample_output_relative_path, natsorted(expected_output_for_relative_paths))

    # test_eq(sample_output_absolute_path, )
    sample_output_name = subsection_folders(mock_index_note, mock_vault, output_type='name')
    test_shuffled(sample_output_name, folders)
    test_eq(sample_output_name, natsorted(folders))

## Corresponding headings in index notes and subfolders

In [ ]:
#| export 
def get_alphanumeric(
        title: str, # The title of either a folder or a heading. Must start with an alphanumeric.
        title_type: str # Either `folder` or `heading`.
        ) -> str: # An alphabet or a numeric (arabic or roman)
    """
    Get the alphanumeric of a title of either a folder or a heading
    in an index noteh.

    Assumes that each folder is titled
    `'{alphanumeric}_{folder_title}'` and each heading is titled
    `'{alphanumeric}. {heading_title}'`
    """
    assert title_type in ['folder', 'heading']
    if title_type == 'folder':
        return re.sub(r'(.*?)_.*' , r'\1', title)
    else:
        return re.sub(r'(.*?)\. .*', r'\1', title)
    


In [ ]:
test_eq(get_alphanumeric('1. Higher direct images', 'heading'), '1')
test_eq(get_alphanumeric('1_higher_direct_images', 'folder'), '1')
test_eq(get_alphanumeric('12_higher_direct_images_the_leray_spectral_sequence', 'folder'), '12')
test_eq(get_alphanumeric('VII_elliptic_curves_over_local_fields', 'folder'), 'VII')
test_eq(get_alphanumeric('A_properties_of_morphisms', 'folder'), 'A')

In [ ]:
#| export 
def correspond_headings_with_folder(
        index_note: VaultNote,
        vault: PathLike,
        include_non_heading: bool = True # If `True`, and if there is text before any heading, then treat such text as being under a "blank" heading.
        ) -> dict[str, str]:
    """
    Return tuples of corresponding headings in an index note
    with folder names.
    
    Assumes that each folder is titled
    `'{alphanumeric}_{folder_title}'` and each heading is titled
    `'{alphanumeric}. {heading_title}'`
    
    **Returns**
    - dict[str, str]
        - Each key is a str indexing the headings and folders. The keys
        are usually alphanumerics (arabic or roman), depending on the
        numbering system of chapters/sections of the reference/text.
        The values are tuples `(folder_title, heading_title)` without 
        the alphanumeric. For the blank heading, the key/index, the folder title,
        and the heading title are all the empty str.
    """
    index = MarkdownFile.from_vault_note(index_note)
    headings = index.get_headings(levels=1)
    headings = [heading_title(heading) for heading in headings]
    folders = subsection_folders(index_note, vault, output_type='name')
    correspond_dict = {get_alphanumeric(heading, 'heading'): (heading, folder)
                       for heading, folder in zip(headings, folders)}
    # TODO do a better job at the conditional below; 
    # for example, consider the start of the text blank if it's just empty lines with spaces.
    if (include_non_heading and index.parts
            and index.parts[0]['type'] != MarkdownLineEnum.HEADING):
        correspond_dict[''] = ('', '')
    return correspond_dict
    

In [ ]:
with (tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir):
    temp_vault = Path(temp_dir) / 'test_vault_2'
    shutil.copytree(_test_directory() / 'test_vault_2', temp_vault)

    two_chapter_reference_1_index_note = VaultNote(temp_vault, name='_index_2_chapter_reference_1')
    test_eq(len(correspond_headings_with_folder(two_chapter_reference_1_index_note, temp_vault)), 1)

In [ ]:
mock_vault = Path('mock_absolute_path')
mock_path =  mock_vault / Path('algebraic_geometry') / Path('some_reference') / Path('chapter_18_some_chapter')
folders = ['181_some_title',
    '182_some_other_title',
    '183_yet_another_title']
mock_glob_return_value = [str(mock_path / folder) for folder in folders]

text = r"""# 18.1. Some title 
- [ ] [[some_reference 18.1|some_reference_some_alias]], 18.1, Page 300
# 18.2. Some other title 
- [ ] [[some_reference 18.2]], 18.2, Page 305
# 18.3. Yet another title 
- [ ] [[some_reference 18.3|]], 18.3, Page 308
"""
mock_index_file = MarkdownFile.from_string(text)

with (mock.patch("__main__.glob.glob", return_value=mock_glob_return_value),
      mock.patch("trouver.markdown.markdown.file.MarkdownFile.from_vault_note", return_value=mock_index_file)):

    mock_index_note = VaultNote(rel_path = '_index_mock.md', vault=mock_vault)
    # subsections_listed_in_index_note(mock_index_note, vault=mock_vault)
    sample_output = correspond_headings_with_folder(mock_index_note, mock_vault)
    print(sample_output)
    test_eq(len(sample_output), 3)
    for key, value in sample_output.items():
        assert value[0].startswith(key)
        assert value[1].startswith(key.replace('.', ''))

{'18.1': ('18.1. Some title', '181_some_title'), '18.2': ('18.2. Some other title', '182_some_other_title'), '18.3': ('18.3. Yet another title', '183_yet_another_title')}


## Move information notes to their appropriate folders.
Sometimes, I end up creating information notes in the wrong folders. It would be nice to detect which ones are in the wrong folders and to move them appropriately.

In [ ]:
#| export
def information_notes_linked_in_index_note(
        index_note: VaultNote, # The note indexing the information notes.
        vault: PathLike,
        hints: list[PathLike] = None # Hints on where the information notes are likely to be found at.  Each path is relative to `vault` and points to a directory. Defaults to `None`.
        ) -> dict[str, list[VaultNote]]: # Each key is the index for the heading (usually either an alphanumerical or a roman numerical). Each value is a list of the information notes linked in the index note.
    """Find information notes to be moved to the correct folder.
    
    Current implementation just looks at level 1 headings.
    This function is used in `move_information_notes_to_correct_folder`.
    Assumes that all notes in the vault have unique names.
    """
    parent_folder = os.path.dirname(index_note.rel_path)
    headings_folders = correspond_headings_with_folder(index_note, vault)
    mf = MarkdownFile.from_vault_note(index_note)
    headings_text = mf.get_headings_and_text(levels=1, include_start=True)
    headings_text = {heading_title(heading): text for heading, text
                     in headings_text.items()}
    text_under_headings = {heading_index: headings_text[heading] 
                            for heading_index, (heading, _) in headings_folders.items()}
    links_by_headings = {heading_index:links_from_text(text) for
                         heading_index, text in text_under_headings.items()}
    note_names_by_headings = {heading_index:[il.file_name for il in links] 
                              for heading_index, links in links_by_headings.items()}
    # Find notes by headings, but also pass the folder corresponding to the heading
    # as a hint of where to find the note for speedup, in case the note is 
    # already at the right place.
    folders_by_index = {heading_index: Path(vault) / parent_folder / heading_folder 
                        for heading_index, (_, heading_folder) in headings_folders.items()}
    if not hints:
        hints = []
    notes_by_headings = {heading_index: [VaultNote(vault, name=nn, hints=hints+[folders_by_index[heading_index]]) 
                                         for nn in note_names]
                         for heading_index, note_names in note_names_by_headings.items()}
    return notes_by_headings
    

In [ ]:
VaultNote.clear_cache()

mock_vault = Path('mock_absolute_path')
mock_path =  mock_vault / Path('algebraic_geometry') / Path('some_reference') / Path('chapter_18_some_chapter')
folders = ['181_some_title',
    '182_some_other_title',
    '183_yet_another_title']
mock_glob_return_value = [str(mock_path / folder) for folder in folders]

mock_correspond_headings_with_folder_return_value = {
  '18.1': ('18.1. Some title', '181_some_title'),
  '18.2': ('18.2. Some other title', '182_some_other_title'),
  '18.3': ('18.3. Yet another title', '183_yet_another_title')}

text = r"""# 18.1. Some title 
- [ ] [[some_reference 18.1|some_reference_some_alias]], 18.1, Page 300
- [ ] [[some_reference 18.1.1|another_alias]], 18.1.1, Page 300
- [ ] [[some_reference 18.1.2]], 18.1.2, Page 301
# 18.2. Some other title 
- [ ] [[some_reference 18.2]], 18.2, Page 305
- [ ] [[some_reference 18.2.1]], 18.2.1, Page 306
# 18.3. Yet another title 
- [ ] [[some_reference 18.3]], 18.3, Page 308
- [ ] [[some_reference 18.3.1]], 18.3.1, Page 308
"""

mock_index_file = MarkdownFile.from_string(text)

mock_index_note = VaultNote(rel_path = mock_path / '_index_18_some_index_note.md', vault=mock_vault)

with (mock.patch("trouver.markdown.markdown.file.MarkdownFile.from_vault_note", return_value=mock_index_file),
      mock.patch("__main__.correspond_headings_with_folder", return_value=mock_correspond_headings_with_folder_return_value),
      # mock.patch("__main__.VaultNote", side_effect=[None, None, None, None, None, None, None])
      ):
    sample_output = information_notes_linked_in_index_note(mock_index_note, mock_vault)
    print(sample_output)
    
    test_eq(len(sample_output), 3)
    test_eq(len(sample_output['18.1']), 3)
    test_eq(len(sample_output['18.2']), 2)
    test_eq(sample_output['18.1'][0].name, 'some_reference 18.1')
    test_eq(sample_output['18.2'][1].name, 'some_reference 18.2.1')


{'18.1': [<trouver.markdown.obsidian.vault.VaultNote object>, <trouver.markdown.obsidian.vault.VaultNote object>, <trouver.markdown.obsidian.vault.VaultNote object>], '18.2': [<trouver.markdown.obsidian.vault.VaultNote object>, <trouver.markdown.obsidian.vault.VaultNote object>], '18.3': [<trouver.markdown.obsidian.vault.VaultNote object>, <trouver.markdown.obsidian.vault.VaultNote object>]}


In [ ]:
#| export    
def move_information_notes_to_correct_folder(
        index_note: VaultNote,
        vault: PathLike,
        hints: list[PathLike] = None # Hints on where the information notes are likely to be found at.  Each path is relative to `vault` and points to a directory. Defaults to `None`.
        ) -> None:
    """Moves the information notes indexed by `index_note` to the correct folder.

    The "correct folder" is a folder in the same directory as `index_note`
    corresponding to the heading under which the information note is indexed.
    The current implementation just looks at level 1 headings.
    """
    parent_folder = os.path.dirname(index_note.path(relative=True))
    linked_notes = information_notes_linked_in_index_note(index_note, vault, hints)
    headings_folders = correspond_headings_with_folder(index_note, vault)
    for heading_index, notes in linked_notes.items():
        _move_notes_under_heading(heading_index, notes,
                                  parent_folder, headings_folders)


def _move_notes_under_heading(
        heading_index, notes: list[VaultNote], parent_folder, headings_folders):
    destination_folder = headings_folders[heading_index][1]
    for note in notes:
        note_folder = os.path.dirname(note.rel_path)
        if destination_folder == note_folder:
            continue
        note.move_to_folder(Path(parent_folder) / destination_folder)


In the following example, there is an index note listing links to three notes. One of the notes does not belong beneath a heading and the other two notes belong beneath a heading. Correspondingly, the `move_information_notes_to_correct_folder` function moves the file for the first note to the same directory that the index note itself is in and the two other notes end up in the section folder corresponding to the heading

In [ ]:
with (tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir,
      ):
    temp_vault = Path(temp_dir) / 'test_vault_2'
    shutil.copytree(_test_directory() / 'test_vault_2', temp_vault)

    index_note = VaultNote(temp_vault, name='_index_1_chapter_reference_4')
    print(f'The following is the text in the index note:\n\n{index_note.text()}')
    move_information_notes_to_correct_folder(index_note, temp_vault)
    assert str(VaultNote(temp_vault, name='this_note_stay_in_the_section_folder').path().parent).endswith('1_section')
    assert str(VaultNote(temp_vault, name='this_note_should_be_moved_to_the_chapter_folder_from_the_section_folder').path().parent).endswith('1_chapter_reference_4')
    assert str(VaultNote(temp_vault, name='this_note_should_be_moved_to_the_section_folder_from_the_chapter_folder').path().parent).endswith('1_section')



The following is the text in the index note:

- [[this_note_should_be_moved_to_the_chapter_folder_from_the_section_folder]]
# 1. First section in 1_chapter_reference_3
- [[this_note_should_be_moved_to_the_section_folder_from_the_chapter_folder]]
- [[this_note_stay_in_the_section_folder]]


In [ ]:
# TODO: see if I can implement the functionalities specified by the following example
# In the following example, there is an unnamed heading not corresponding to any section and not corresponding to any folder within the directory that the index note belongs to. The notes linked beneath the unnamed heading should nevertheless be moved into the directory that the index note is in.

# with (tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir,
#       ):
#     temp_vault = Path(temp_dir) / 'test_vault_2'
#     shutil.copytree(_test_directory() / 'test_vault_2', temp_vault)

#     index_note = VaultNote(temp_vault, name='_index_3_chapter_reference_4')
#     print(f'The following is the text in the index note:\n\n{index_note.text()}')
#     move_information_notes_to_correct_folder(index_note, temp_vault)
#     note_that_should_have_moved = VaultNote(temp_vault, name='this_note_is_in_the_temp_folder_and_should_be_moved_3_chapter_reference_4_where_it_is_linked_by_the_index_note')
#     assert str(note_that_should_have_moved.path().parent).endswith('3_chapter_reference_4')
#     # assert str(VaultNote(temp_vault, name='this_note_should_be_moved_to_the_chapter_folder_from_the_section_folder').path().parent).endswith('1_chapter_reference_4')
#     # assert str(VaultNote(temp_vault, name='this_note_should_be_moved_to_the_section_folder_from_the_chapter_folder').path().parent).endswith('1_section')

In [ ]:
#| export
def move_information_notes_to_correct_folder_for_all_indices(
        index_of_index_notes: VaultNote, # The index note indexing other index notes; `index_of_index_notes` is intended to be an index note for an entire reference whereas the index notes are intended to correspond to chapters/sections in the reference.
        vault: PathLike,
        hints: list[PathLike] = [] # Hints on where the information notes are likely to be found at.  Each path is relative to `vault` and points to a directory.
        ) -> None:
    """
    Moves the information notes for all index notes belonging to the reference as
    specified by `index_of_index_notes`.
    """
    index_of_index_file = MarkdownFile.from_vault_note(index_of_index_notes)
    text = str(index_of_index_file)
    index_files = link_ranges_in_text(text)
    index_files = [ObsidianLink.from_text(text[start:end])
                   for start, end in index_files]
    for link in index_files:
        index_note_name = link.file_name
        index_note = VaultNote(vault, name=index_note_name)
        move_information_notes_to_correct_folder(
            index_note, vault, hints=hints+[index_note.rel_path])

The following example demonstrating `move_information_notes_to_correct_folder` and `move_information_notes_to_correct_folder_for_all_indices` concerns `test_vault_2` in `nbs/_tests`.

Note that it contains the note `_index_1_chapter_reference_1`, which has the following content:


In [ ]:
vn = VaultNote(_test_directory(), name='_index_1_chapter_reference_1')
print(vn.text())

# 1. Section
- [[note_11]]
- [[note_12]]
- [[note_13]]

# 2. Section
- [[note_21]]
- [[note_22]]

# 3. Section
- [[note_31]]
- [[note_32]]
- [[a_note_belonging_in_3_section_1_chapter_reference_1]]

# 4. Section
- [[note_41]]
- [[note_42]]


However, the following notes are in the "wrong" folders according to these index notes:

- `note_21.md` is in the folder `1_section_1_chapter_reference_1`, but it should be in the folder `2_section_1_chapter_reference_1`.
- `note_41.md` is in the folder `3_section_1_chapter_reference_1`, but it should be in the folder `4_section_1_chapter_reference_1`.
- `note_42.md` is in the folder `3_section_1_chapter_reference_1`, but it should be in the folder `4_section_1_chapter_reference_1`.
- `a_note_belonging_in_1_section_1_chapter_reference_2.md` is in the folder `4_section_1_chapter_reference_1`, but it should be in the folder `1_section_1_chapter_reference_2`.
- `a_note_belonging_in_1_section_2_chapter_reference_1.md` is in the folder `4_section_1_chapter_reference_1`, but it should be in the folder `1_section_2_chapter_reference_1`.
- `a_note_belonging_in_3_section_1_chapter_reference_1.md` is in the folder `1_section_2_chapter_reference_2`, but it should be in the folder `3_section_1_chapter_reference_1`.




The `move_information_notes_to_correct_folder` method first applied to the index note `_index_1_chapter_reference_1.md` moves the notes indexed in the index note to their correct locations. In particular, the following notes are moved to their correct locations:

- `note_21.md`,
- `note_41.md`,
- `note_42.md`, and
- `a_note_belonging_in_3_section_1_chapter_reference_1.md`

The `move_information_notes_to_correct_folder` method applied to `_index_2_chapter_reference_1.md` then moves `a_note_belonging_in_1_section_2_chapter_reference_1.md` to its correct location.

Lastly, the `move_information_notes_to_correct_folder` method applied to `_index_1_chapter_reference_2.md` then moves `a_note_belonging_in_1_section_1_chapter_reference_2.md` to its correct location.


In [ ]:
#| hide

# GitHub Actions was not passing the example in the next cell by virtue of not
# moving the note # `a_note_belonging_in_1_section_2_chapter_reference_1`
# to its correct location.
# The below mock test was written to find out why this is happening.
with (tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir,
      mock.patch('__main__.VaultNote.move_to_folder') as mock_vaultnote_move_to_folder):
    temp_vault = Path(temp_dir) / 'test_vault_2'
    shutil.copytree(_test_directory() / 'test_vault_2', temp_vault)

    two_chapter_reference_1_index_note = VaultNote(temp_vault, name='_index_2_chapter_reference_1')

    note_before_moving = VaultNote(temp_vault, name='a_note_belonging_in_1_section_2_chapter_reference_1')
    test_eq(path_name_no_ext(note_before_moving.path().parent), '4_section_1_chapter_reference_1')
    move_information_notes_to_correct_folder(two_chapter_reference_1_index_note, temp_vault)
    # mock_vaultnote_move_to_folder.assert_called()
    # mock_vaultnote_move_to_folder.assert_called_with(
    #   Path('folder_1') / 'reference_1' / '2_chapter_reference_1' / '1_section_2_chapter_reference_1')

# The `VaultNote` object's `move_to_folder` method was not being called in
# GitHub Actions. This happened because the
# `information_notes_linked_in_index_note` function found 
# no linked notes in the index note that are found within
# the invocation of # `move_information_notes_to_correct_folder` 
#
# The below was written to test whether the `move_to_folder` method
# would be called correctly if I mock the
# `information_notes_linked_in_index_note`:


with (tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir,
      mock.patch('__main__.information_notes_linked_in_index_note') as mock_linked_notes,
      mock.patch('__main__.VaultNote.move_to_folder') as mock_vaultnote_move_to_folder):
    temp_vault = Path(temp_dir) / 'test_vault_2'
    shutil.copytree(_test_directory() / 'test_vault_2', temp_vault)

    two_chapter_reference_1_index_note = VaultNote(temp_vault, name='_index_2_chapter_reference_1')

    note_before_moving = VaultNote(temp_vault, name='a_note_belonging_in_1_section_2_chapter_reference_1')
    test_eq(path_name_no_ext(note_before_moving.path().parent), '4_section_1_chapter_reference_1')
    mock_linked_notes.return_value = {'1': [note_before_moving]} 
    move_information_notes_to_correct_folder(two_chapter_reference_1_index_note, temp_vault)
    mock_vaultnote_move_to_folder.assert_called()
    mock_vaultnote_move_to_folder.assert_called_with(
       Path('folder_1') / 'reference_1' / '2_chapter_reference_1' / '1_section_2_chapter_reference_1')

# Ultimately, it turned out that the error was happening because git does not 
# track empty folders and I had an empty folder in my test vault.

In [ ]:
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir:
    temp_vault = Path(temp_dir) / 'test_vault_2'
    shutil.copytree(_test_directory() / 'test_vault_2', temp_vault)
    # os.startfile(temp_dir)
    # input()

    one_chapter_reference_1_index_note = VaultNote(temp_vault, name='_index_1_chapter_reference_1')
    two_chapter_reference_1_index_note = VaultNote(temp_vault, name='_index_2_chapter_reference_1')
    one_chapter_reference_2_index_note = VaultNote(temp_vault, name='_index_1_chapter_reference_2')

    move_information_notes_to_correct_folder(one_chapter_reference_1_index_note, temp_vault)
    note_21 = VaultNote(temp_vault, name='note_21')
    test_eq(path_name_no_ext(note_21.path().parent), '2_section_1_chapter_reference_1')
    note_41 = VaultNote(temp_vault, name='note_41')
    test_eq(path_name_no_ext(note_41.path().parent), '4_section_1_chapter_reference_1')
    note_42 = VaultNote(temp_vault, name='note_42')
    test_eq(path_name_no_ext(note_42.path().parent), '4_section_1_chapter_reference_1')
    note_alpha = VaultNote(temp_vault, name='a_note_belonging_in_3_section_1_chapter_reference_1')
    test_eq(path_name_no_ext(note_alpha.path().parent), '3_section_1_chapter_reference_1')

    # os.startfile(temp_vault)
    move_information_notes_to_correct_folder(two_chapter_reference_1_index_note, temp_vault)
    note_beta = VaultNote(temp_vault, name='a_note_belonging_in_1_section_2_chapter_reference_1')
    test_eq(path_name_no_ext(note_beta.path().parent), '1_section_2_chapter_reference_1')

    move_information_notes_to_correct_folder(one_chapter_reference_2_index_note, temp_vault)
    note_gamma = VaultNote(temp_vault, name='a_note_belonging_in_1_section_1_chapter_reference_2')
    test_eq(path_name_no_ext(note_gamma.path().parent), '1_section_1_chapter_reference_2')
    

The `move_information_notes_to_correct_folder_for_all_indices` applied to `_index_reference_1.md` effectively invokes `move_information_notes_to_correct_folder` to the index notes which are indexed in `_index_reference_1.md`, i.e. to `_index_1_chapter_reference_1`, `_index_2_chapter_reference_1`, `_index_3_chapter_reference_1`. Note that this invocation of the method does not invoke `move_information_notes_to_correct_folder` to `_index_1_chapter_reference_2` and hence `a_note_belonging_in_1_section_1_chapter_reference_2` is not moved to its correct location.

In [ ]:
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir:
    temp_vault = Path(temp_dir) / 'test_vault_2'
    shutil.copytree(_test_directory() / 'test_vault_2', temp_vault)

    reference_1_index_note = VaultNote(temp_vault, name='_index_reference_1')
    move_information_notes_to_correct_folder_for_all_indices(reference_1_index_note, temp_vault)

    # These notes are moved to their correct locations because they belong to reference_1, i.e. 
    # are indexed in indexed notes which are indexed in `_index_reference_1`.
    note_21 = VaultNote(temp_vault, name='note_21')
    test_eq(path_name_no_ext(note_21.path().parent), '2_section_1_chapter_reference_1')
    note_41 = VaultNote(temp_vault, name='note_41')
    test_eq(path_name_no_ext(note_41.path().parent), '4_section_1_chapter_reference_1')
    note_42 = VaultNote(temp_vault, name='note_42')
    test_eq(path_name_no_ext(note_42.path().parent), '4_section_1_chapter_reference_1')
    note_alpha = VaultNote(temp_vault, name='a_note_belonging_in_3_section_1_chapter_reference_1')
    test_eq(path_name_no_ext(note_alpha.path().parent), '3_section_1_chapter_reference_1')

    note_beta = VaultNote(temp_vault, name='a_note_belonging_in_1_section_2_chapter_reference_1')
    test_eq(path_name_no_ext(note_beta.path().parent), '1_section_2_chapter_reference_1')

    # This note is not moved to its correct location because it does not belong to reference_2.
    note_gamma = VaultNote(temp_vault, name='a_note_belonging_in_1_section_1_chapter_reference_2')
    test_ne(path_name_no_ext(note_gamma.path().parent), '1_section_1_chapter_reference_2')
    test_eq(path_name_no_ext(note_gamma.path().parent), '4_section_1_chapter_reference_1')



## Automatically make subfolders based on index note headings

In [ ]:
#| export
def convert_title_to_folder_name(title: str) -> str:
    # TODO: remove left/right
    """
    Returns a folder name for the given string, e.g. replaces spaces
    with underscore.
    
    **Parameters**
    - `title` - str
    
    **Returns**
    - str
    """
    characters_to_remove = [
        '.', '\'', '$', '(', ')', '{', '}', ':', '?', '!', ',', '#', '%', '&',
        '\\', '<', '>', '*', '?', '/', '"', '@', '+', '`', '|', '=', '[', ']',
        'mathscr', 'mathbf', 'mathrm', ]
    characters_to_turn_to_underscore = [' ', '-', '^']
    title = title.strip()
    title = title.lower()
    for character in characters_to_remove:
        title = title.replace(character, '')
    for character in characters_to_turn_to_underscore:
        title = title.replace(character, '_')
    title = sanitize_filename(title)
    return title

In [ ]:
# TODO: add examples
sample_title = convert_title_to_folder_name(r'1. $\mathscr{M}_g$ and its boundary')
print(sample_title)

sample_title = convert_title_to_folder_name(r'''7. Exceptional maximal subgroups of 
\texorpdfstring{\(\GSp_4(\ff_\ell)\)}{GSp4Fell}''')
print(sample_title)
validate_filename(sample_title)

1_m_g_and_its_boundary
7_exceptional_maximal_subgroups_of_texorpdfstringgsp_4ff_ellgsp4fell


In [ ]:
#| export
def convert_heading_to_folder_name(
        heading: str # Matches regex `\# (\w+?)\. (.*?)`
        ) -> str:
    """Converts a heading to a valid name for a folder.
    
    TODO Might not work correctly.

    **Parameters**
    - heading: str
        
    """
    regex_match = re.match(r'\# (\w+?)\. (.*)', heading)
    try:
        alphanumeric = regex_match.group(1)
        title = regex_match.group(2)
    except AttributeError:
        raise ValueError(
            f"`convert_heading_to_folder_name` unsuccessfully attempted"
            f" to match the following: {heading}")
        #print(heading)
    return f'{alphanumeric}_{convert_title_to_folder_name(title)}'    

In [ ]:
print(convert_heading_to_folder_name(r'# 1. First title'))
print(convert_heading_to_folder_name(r'# A. appendix title'))
# print(convert_heading_to_folder_name(r'# A.1. appendix title')) # Works incorrectly TODO fix
print(convert_heading_to_folder_name(r"# 1. Hi I'm Bob"))

1_first_title
A_appendix_title
1_hi_im_bob


In [ ]:
#| export
def make_folders_from_index_note_headers(
        index_note: VaultNote
        ) -> None:
    r"""
    Make folders in the same directory as index note whose names
    are the titles of the headers of the index note.

    The headers of the index note must match the regex pattern `\# (\w+?)\. (.*?)`.
    """
    mfile = MarkdownFile.from_vault_note(index_note)
    headings = mfile.get_headings_by_line_number(levels=1)
    pattern = re.compile(r'\# (\w+?)\. (.*?)')
    folder_names = [convert_heading_to_folder_name(heading)
                    for _, heading in headings.items() if heading]
    directory = Path(os.path.dirname(index_note.path()))
    for folder_name in folder_names:
        try:
            os.mkdir(directory / folder_name)
        except OSError as error:
            pass

In [ ]:
with (tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir,
      mock.patch('__main__.convert_heading_to_folder_name') as mock_convert_heading_to_folder_name):
    temp_vault = Path(temp_dir) / 'test_vault_2'
    shutil.copytree(_test_directory() / 'test_vault_2', temp_vault)

    # This mocks return values of `convert_heading_to_folder_name` which is used to
    # determine the names of the folders to be made; I am using this because 1. I 
    # do not really care what the folders are named for the purposes of this example/test
    # and 2. I anticipate that I will make several modifications to the method before
    # I am satisfied with it.
    mock_convert_heading_to_folder_name.side_effect = [
        "1_first_section_in_1_chapter_reference_3",
        "2_second_section_in_1_chapter_reference_3",
        "3_hi_im_bob"]
    
    index_note = VaultNote(temp_vault, name='_index_1_chapter_reference_3')
    make_folders_from_index_note_headers(index_note)
    absolute_path = index_note.path(relative=False)
    index_note_directory = absolute_path.parent
    list_of_folders_in_the_same_directory_as_index_note = os.listdir(
        index_note_directory)

    # 3 folders plus the index note itself will be made in the 
    test_eq(len(list_of_folders_in_the_same_directory_as_index_note), 4)

    mock_convert_heading_to_folder_name.assert_has_calls(
        [mock.call('# 1. First section in 1_chapter_reference_3'),
         mock.call('# 2. Second section in 1_chapter_reference_3'),
         mock.call("# 3. Hi I'm Bob")])

## Identify order of notes in index notes

In [ ]:
#| export
# TODO: do an example of the `include_embedded_notes` paramtere.
def get_notes_from_index_note(
        vault: PathLike, # The path to the Obsidian vault directory
        index_note: VaultNote, # The VaultNote object for the index note.
        as_vault_notes: bool = True, # If `True`, returns the ``VaultNote`` objects for the index notes. Otherwise, returns the names of these notes 
        include_embedded_notes: bool = False # If `True`, include in the list the embedded notes. Defaults to `False`.
        ) -> list[Union[str, VaultNote]]: # Either of the names of the index notes in the vault or of the index notes as VaultNote objects, depending on `as_vault_notes`.
    """Returns the list of notes listed in the index note in the order that
    they are listed in.
    
    Asssumes that the index note is "formatted correctly".
    
    **See Also**
    - ``get_index_notes_from_index_note`` in 
    ``markdown.obsidian.personal.reference``.
    """
    vault = Path(vault)
    text = index_note.text()
    links = links_from_text(text)
    if not include_embedded_notes:
        links = [link for link in links if not link.is_embedded]
    index_notes = [link.file_name for link in links]
    if as_vault_notes:
        index_notes = [VaultNote(vault, name=index_note)
                       for index_note in index_notes]
    return index_notes

We can get the notes indexed in an index note in the order that they are indexed:

In [ ]:
vault = _test_directory() / 'test_vault_2'
index_note = VaultNote(vault, name='_index_1_chapter_reference_1')
list_of_notes_in_index_note = get_notes_from_index_note(vault, index_note)
test_eq(len(list_of_notes_in_index_note), 10)
names_of_notes_in_index_note = [vn.name for vn in list_of_notes_in_index_note]
print(names_of_notes_in_index_note)
test_eq(names_of_notes_in_index_note[0], "note_11")
test_eq(names_of_notes_in_index_note[7], "a_note_belonging_in_3_section_1_chapter_reference_1")

['note_11', 'note_12', 'note_13', 'note_21', 'note_22', 'note_31', 'note_32', 'a_note_belonging_in_3_section_1_chapter_reference_1', 'note_41', 'note_42']


We can just get the names of these notes instead of `VaultNote` objects representing these notes:

In [ ]:
list_of_notes_in_index_note = get_notes_from_index_note(vault, index_note, as_vault_notes=False)
test_eq(len(list_of_notes_in_index_note), 10)
print(list_of_notes_in_index_note)

['note_11', 'note_12', 'note_13', 'note_21', 'note_22', 'note_31', 'note_32', 'a_note_belonging_in_3_section_1_chapter_reference_1', 'note_41', 'note_42']


## Add line in index note

In [ ]:
#| export
def add_link_in_index_note_after_note_link(
        index_note: VaultNote,
        note_to_add_link_after: VaultNote,
        link_to_add: ObsidianLink) -> None:
    """
    Adds a link in the index note.

    The link is added after the first link to `note_to_add_link_after`. 
    If no link to `note_to_add_link_after` is found, then a link is added
    at the end.
    """
    link_to_find = ObsidianLink(False, note_to_add_link_after.name, -1, -1)
    pattern = link_to_find.to_regex()
    mf = MarkdownFile.from_vault_note(index_note)
    for i, part in enumerate(mf.parts):
        if re.search(pattern, part['line']):
            break
    mf.insert_line(i + 1, {'type': MarkdownLineEnum.UNORDERED_LIST,
                           'line': f'- {link_to_add.to_string()}'})
    mf.write(index_note)

In [ ]:
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir:
    temp_vault = Path(temp_dir) / 'test_vault_2'
    shutil.copytree(_test_directory() / 'test_vault_2', temp_vault)

    index_note = VaultNote(temp_vault, name='_index_1_chapter_reference_1')
    note_to_add_link_after = VaultNote(temp_vault, name='note_21')
    link_to_add = ObsidianLink.from_text('[[another_note]]')
    add_link_in_index_note_after_note_link(index_note, note_to_add_link_after, link_to_add)
    new_text_of_the_index_note = index_note.text()
    assert str(link_to_add) in new_text_of_the_index_note
    print(new_text_of_the_index_note)

# 1. Section
- [[note_11]]
- [[note_12]]
- [[note_13]]

# 2. Section
- [[note_21]]
- [[another_note]]
- [[note_22]]

# 3. Section
- [[note_31]]
- [[note_32]]
- [[a_note_belonging_in_3_section_1_chapter_reference_1]]

# 4. Section
- [[note_41]]
- [[note_42]]


If there are multiple instances of `note_to_add_link_after`, then the link is added after the first link to `note_to_add_link_after`.

In [ ]:
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir:
    temp_vault = Path(temp_dir) / 'test_vault_2'
    shutil.copytree(_test_directory() / 'test_vault_2', temp_vault)

    index_note = VaultNote(temp_vault, name='_index_2_chapter_reference_3')
    note_1 = VaultNote(temp_vault, name='link_1')
    link_to_add = ObsidianLink(False, 'new_link', 0, 0)
    add_link_in_index_note_after_note_link(index_note, note_1, link_to_add)

    new_text_of_the_index_note = index_note.text()
    assert new_text_of_the_index_note.startswith('- [[link_1]]\n- [[new_link]]')
    assert str(link_to_add) in new_text_of_the_index_note
    print(new_text_of_the_index_note)

- [[link_1]]
- [[new_link]]
- [[link_2]]
- [[link_3]]
- [[link_1]]


If there are no links to `note_to_add_link_after`, then the link is added at the end. 

In [ ]:
# TODO: do example.
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir:
    temp_vault = Path(temp_dir) / 'test_vault_2'
    shutil.copytree(_test_directory() / 'test_vault_2', temp_vault)

    index_note = VaultNote(temp_vault, name='_index_2_chapter_reference_3')
    note_1 = VaultNote(temp_vault, name='link_that_does_not_exist')
    link_to_add = ObsidianLink(False, 'new_link', 0, 0)
    add_link_in_index_note_after_note_link(index_note, note_1, link_to_add)

    new_text_of_the_index_note = index_note.text()
    print(new_text_of_the_index_note)
    assert new_text_of_the_index_note.endswith('- [[link_1]]\n- [[new_link]]')
    assert str(link_to_add) in new_text_of_the_index_note

- [[link_1]]
- [[link_2]]
- [[link_3]]
- [[link_1]]
- [[new_link]]
